## Summary Data for GraphML Files

In [8]:
import graph_tool.all as gt
import os
import numpy as np
import graph_tool
from tqdm import tqdm_notebook

from lib.analysis import clustering
from lib.analysis import centrality
from lib.analysis import dist
from lib.analysis import model
from lib.analysis import power
from lib.analysis import stats

## Part 1: Summary Statistics

Basic statistics for each country's government

In [ ]:
data = 'data'
ignore = ['.DS_Store']
nets = {}

In [13]:
for country in [d for d in os.listdir(data) if d not in ignore]:
    path = os.path.join(data, country)
    if country not in nets:
        nets[country] = []
    for dir_ in [f for f in os.listdir(path) if f not in ignore]:
        if not dir_.endswith('.graphml'):
            net_path = os.path.join(path, dir_)
            for net in [f for f in os.listdir(net_path) if f not in ignore]:
                if net.endswith('.graphml'):
                    nets[country].append((net[:-8], gt.load_graph(os.path.join(net_path, net))))

In [14]:
for country, graphs in nets.items():
    with open(f'summary.{country}.csv', 'w+') as f:
        f.write(','.join(['Name', 'Number of Edges', 'Number of Vertices',
                  'Clustering Coefficient', 'Average Degree',
                  'Average Excess Degree', 'Average Distance',
                  'Size of LCC', 'Number of CC']) + '\n')
        for name, graph in graphs:
            f.write(f'{name},{str(stats.summary(graph))}\n')

## Part 2: Assortivity

Computes assoritivty on specified attribute for every country across years 

In [13]:
individual_graphs = {}
root = 'data/'
countries = os.listdir(root)
nets = [country + "/" + net + "/" for country in countries
                            for net in os.listdir(root + country)
                            if not net.endswith('.graphml') and net != '.DS_Store']
for net in nets:
    print(net)
    for x in os.listdir(root+net):
        if x != '.DS_Store' and x.endswith('.graphml') and x != 'raw':
            individual_graphs[x] = gt.load_graph(root + net + x) 

sweden/net_se/
france/net_fr_an/
france/net_fr_se/
switzerland/net_ch_cs/
switzerland/net_ch_cn/
uk/graphs/
uk/raw/
denmark/net_dk/
austria/net_at/


In [15]:
attribute = 'party'
label = 'party_id'
with open("assortivity_summary.csv", "w+") as f:
    f.write(','.join(['Assortivity Measure']) + '\n')
    for name, g in individual_graphs.items():
        if attribute in g.vertex_properties:
            print(name, attribute)
            g.set_directed(False)
            gt.remove_parallel_edges(g)
            stats.generate_party_property(g, attribute)
            f.write(f'{name},{str(stats.assortativity_summary(g, label))}\n')


net_se2014-2018.graphml party
net_se2010-2014.graphml party
net_se2006-2010.graphml party
net_se1998-2002.graphml party
net_se2002-2006.graphml party
net_se1988-1991.graphml party
net_se1994-1998.graphml party
net_se1991-1994.graphml party
net_fr_an2007-2012.graphml party
net_fr_an2002-2007.graphml party
net_fr_an1986-1988.graphml party
net_fr_an2012-2017.graphml party
net_fr_an1997-2002.graphml party
net_fr_se1997-2002.graphml party
net_fr_se2012-2017.graphml party
net_fr_se1986-1988.graphml party
net_fr_se1988-1993.graphml party
net_fr_se1993-1997.graphml party
net_fr_se2002-2007.graphml party
net_fr_se2007-2012.graphml party
net_ch_cs1995-1999.graphml party
net_ch_cs2003-2007.graphml party
net_ch_cs2011-2015.graphml party
net_ch_cs1999-2003.graphml party
net_ch_cs2007-2011.graphml party
net_ch_cn2003-2007.graphml party
net_ch_cn1999-2003.graphml party
net_ch_cn2007-2011.graphml party
net_ch_cn2011-2015.graphml party
net_ch_cn1995-1999.graphml party
net_dkEconomy.graphml party
net_dk

In [14]:
for name, g in individual_graphs.items():
    name = name.split('.')[0]
    if label in g.vertex_properties:
        gt.graph_draw(g, vertex_fill_color = g.vertex_properties['party_id'], output = "assortativity_" + name + ".pdf" )

NameError: name 'label' is not defined

## Part 3: Graphing Centrality

In [23]:
# BETWEENNESS
for name, g in individual_graphs.items():
    name = name.split('.')[0]
    if label in g.vertex_properties:
        vertex, edge = centrality.betweenness(g, scale=50)
        gt.graph_draw(g, vertex_size = vertex, edge_pen_width=edge, edge_color=edge, output="edge_betweenness_" + name + ".pdf" )

In [19]:
# CLOSENESS
for name, g in individual_graphs.items():
    name = name.split('.')[0]
    if label in g.vertex_properties:
        vertex = centrality.closeness(g, scale=20)
        gt.graph_draw(g, vertex_size=vertex, vertex_fill_color=vertex, edge_pen_width=0.1, output="vertex_closeness_" + name + ".pdf" )

In [29]:
# EIGENVECTOR vertex_size=vector
for name, g in individual_graphs.items():
    name = name.split('.')[0]
    if label in g.vertex_properties:
        value, vector = centrality.eigenvector(g, scale=70)
        gt.graph_draw(g, vertex_pen_width=1.4, vertex_fill_color=vector, edge_color='#D3D3D3', edge_pen_width=0.1, output="vertex_eigenvector_" + name + ".pdf" )